In [1]:
!pip install transformers==4.44.2
!pip install datasets==2.19.0
!pip install -U bitsandbytes

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
import torch
import pandas as pd
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from tqdm import tqdm
import re
from typing import Dict, List
import os
import json
import yaml
import threading
import sys
import time

In [50]:
def generate(tokenizer, prompt, model, temperature = 1, top_p = 1):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    output_ids = model.generate(
        input_ids, 
        attention_mask=attention_mask,
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=2048,
        pad_token_id=tokenizer.pad_token_id
    )
    output_text = tokenizer.decode(output_ids[0])
    return output_text

Login to Hugging face

In [ ]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

login(token = config["hf_token"])

Insert below the path to the adapters

In [ ]:

# WRITE PATH TO ADAPTERS HERE
ADAPTERS_PATH = "/source/aixpa/checkpoints/Llama-3.1-8B-Instruct/run1/checkpoint-250"

Loading local model

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# loading fine-tuned model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# loading adapters
model = PeftModel.from_pretrained(model, ADAPTERS_PATH)
model = model.merge_and_unload()
model.config.use_cache = True

# loading tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|█████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.11s/it]
/home/land/.local/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


## Convert LLM template to dictionary

In [6]:
def template_to_dict_list(dial_template: str) -> List[Dict[str, str]]:
    """
    Converts llama 3.1's dialogue template to a lift of dictionaries

    The list is formattes as:
    [
        {"role": ROLE1, "content": CONTENT1},
        {"role": ROLE2, "content": CONTENT2},
    ]
    "role" key can be system, user, or assistant 

    NOTE: Adds "document" key to the dictionary if the turn has 
    text in between <document> tags

    Arguments:
        dial_template (str) - the LLama 3.1 to be converted to a dictionary

    Returns:
        (List[Dict[str, str]]) - the reshared dialogue
    """
    raw_turn_list = [dial.replace("<|start_header_id|>", "")\
     .replace("<|begin_of_text|>", "")\
     .split("<|end_header_id|>")
     for dial in dial_template.split("<|eot_id|>")[:-1]]

    # adding everything to dictionary
    turn_list = [{"role": turn[0], "content": turn[1].strip()} for turn in raw_turn_list]


    # adding "document" key
    for turn in turn_list:
        match = re.search(r"<document>(.*?)</document>", turn["content"], re.DOTALL)
        if match:  # Ensure there's a match before calling .group()
            turn["document"] = match.group(1).split("<document>")[1].replace("<\document>", "")

    return turn_list

## Model interaction

In [67]:
spinner_event = threading.Event()

def rotating_spinner(start_time):
    """Displays a rotating spinner with elapsed time while spinner_event is set."""
    spinner = ['|', '/', '-', '\\']
    dots    = ['.', '..', '...']
    cnt     = 0
    while spinner_event.is_set():
        elapsed_time = time.time() - start_time
        symbol   = spinner[cnt % len(spinner)]
        ellipsis = dots[cnt//4 % len(dots)]
        # for symbol in spinner:
        sys.stdout.write(f'\rMODEL: {symbol} Obtaining response{ellipsis.ljust(3, " ")} ({elapsed_time:.1f} seconds)')  # overwrite the same line
        sys.stdout.flush()
        time.sleep(0.15)
        cnt += 1
    sys.stdout.write(f'\rMODEL:   (wait time: {elapsed_time:.1f} seconds)                 \n')  # Clear spinner line

def llama_interactive_dialogue(syst_prompt: str) -> None:
    continue_chat = True
    chat_dict = [{"role": "system", "content": syst_prompt}]
    
    sys.stdout.write("         You are starting a conversation            \n")
    sys.stdout.write("- - - - - - - - - - - - - - - - - - - - - - - - - - \n")
    sys.stdout.write("> to interrupt the chat, write \"stop\" or \"exit\" \n")
    sys.stdout.write("> to save the chat on the disk, write \"save\"      \n")
    sys.stdout.write("- - - - - - - - - - - - - - - - - - - - - - - - - - \n")
    
    while continue_chat:
        user_in = input("\nUSER:\n")
        
        ### Verify if the input changes the chat status
        if user_in.lower() in {"stop", "exit"}:
            print("Chat stopped")
            continue_chat = False
            continue
            
        elif user_in.lower() == "save":
            in_filename = input("Filename: ")
            
            filename  = f"{in_filename}.txt"
            base_path = os.path.join("manual_dialogue_saves/", filename)
            file_path = base_path
            counter = 1

            while os.path.exists(file_path):
                filename  = f"{in_filename}_{counter}.txt"
                file_path = os.path.join("manual_dialogue_saves/", filename)
                counter += 1
            
            with open(file_path, "w") as file:
                json.dump(chat_dict, file, indent=4)
            print(f"Chat saved as: {filename}")
            continue_chat = False
            continue
            
        ### continue the conversation
        chat_dict.append({"role": "user", "content": user_in})
        chat_template = [f"<|start_header_id|>{turn['role']}<|end_header_id|>\n\n{turn['content']}<|eot_id|>" for turn in chat_dict]
        chat_template = "".join(chat_template)
            
        template = "<|begin_of_text|><|start_header_id|>system<|end_header_id|> " + \
            "\n\nCutting Knowledge Date: September 2024\nToday Date: 1 Apr 2024\n\n" + \
            syst_prompt + "<|eot_id|>" + "".join(chat_template)
        
        sys.stdout.write(f'\n')
        
        # Start the spinner
        spinner_event.set()
        spinner_thread = threading.Thread(target=rotating_spinner, args=(time.time(),))
        spinner_thread.start()

        try:
            reply = generate(tokenizer, template, model, temperature = 0.7, top_p=0.9).split("<|end_header_id|>")[-1].strip("\n<|eot_id|>").strip()
        finally:
            # Stop the spinner
            spinner_event.clear()
            spinner_thread.join()

        print(reply)
        chat_dict.append({"role": "assistant", "content": reply})


In [ ]:
interactive_doc = """
=== PIANO FAMIGLIA DI ORBASSANO ANNO 2022 ===

Il Comune di Orbassano, unitamente al Consorzio Intercomunale di Servizi (CIdiS) e ai Comuni consorziati, ha avviato un processo volto a sostenere ulteriormente le politiche per il benessere della comunitа e delle famiglie, per perseguirne la piena promozione, anche attraverso il coinvolgimento delle reti locali e delle risorse attivabili sul territorio comunale. La realizzazione di tale processo presuppone il superamento di logiche assistenzialistiche, in favore di approcci orientati allo sviluppo di comunitа, trasversali e interdisciplinari (servizi sociali, altri servizi, tempo libero, cultura, ambiente, sport, casa, lavoro, trasporti, etc.), che considerano centrale la promozione sociale ed in cui la famiglia diventa soggetto attivo e propositivo. Il CIdiS, con Deliberazione dell’Assemblea Consortile n. 16, in data 16.11.2020, ha approvato l’“Accordo di collaborazione tra CIdiS e Provincia Autonoma di Trento per la promozione di politiche e servizi in materia di benessere delle famiglie e della comunitа”, il cui obiettivo è la realizzazione di un percorso di certificazione territoriale allo scopo di accrescere, tramite il rafforzamento del sistema dei servizi e delle iniziative per la famiglia, l’attrattivitа territoriale, nonchй di sostenere lo sviluppo locale, attraverso il coinvolgimento di tutte le organizzazioni interessate che condividono i medesimi obiettivi. A fronte di quanto sopra richiamato, il Comune di Orbassano, con la Deliberazione di Giunta Comunale n. 40 del 29 marzo 2021, cui si rimanda, ha aderito al “Network Nazionale Comuni Amici della Famiglia” avviando cosм nel proprio territorio detto processo orientato alla realizzazione di politiche per il benessere della comunitа e delle famiglie. Il Comune di Orbassano ha una realtа ricca di associazioni, gruppi informali, societа sportive etc. che a vario titolo si occupano di promuovere iniziative per giovani e famiglie. И, dunque, importante saper cogliere tali opportunitа, metterle in rete e creare sinergie che possano portare beneficio alle singole famiglie e all’intera comunitа. И di particolare interesse per l’Amministrazione comunale avvalersi dello specifico marchio “Comune Amico della Famiglia - Family Italia” rilasciato dall’Agenzia per la Famiglia della Provincia Autonoma di Trento, in quanto certificazione di impegno ad orientare le politiche in un’ottica family friendly, mettendo, dunque, in campo servizi e interventi che rispondano alle esigenze e alle aspettative espresse dalle comunitа locali.

L’attuazione del percorso di certificazione è finalizzata a: 

migliorare la qualitа del benessere territoriale e creare un territorio più forte e attrattivo; 

contraddistinguere l’impegno concreto per garantire lo sviluppo e la diffusione di politiche di benessere; 

consentire alle amministrazioni maggiore visibilitа e riconoscimento in quanto enti che aderiscono al progetto di costruzione di un territorio “a misura di famiglia”;

consentire di sviluppare un marketing territoriale family oriented, ovvero, un territorio capace di offrire servizi ed opportunitа rispondenti alle aspettative della comunitа; 

contrastare il calo demografico attraverso politiche che assicurino la piena promozione della famiglia e il sostegno nell’esercizio delle proprie funzioni educative e sociali.

Nel dicembre 2020 si è condiviso con l’Agenzia per la Famiglia della Provincia Autonoma di Trento il piano delle azioni che le parti stanno realizzando nel corso del 2021, in coerenza con gli impegni assunti nell’ambito del citato accordo di collaborazione, articolato in 4 macro-azioni di seguito descritte:

nuovo management pubblico familiare

certificazione family audit

comuni “Amici della Famiglia”

sussidiarietа organizzativa.

A maggio 2021 è stato istituito il “Tavolo Intercomunale per la promozione di politiche e servizi in materia di benessere delle famiglie e della comunitа”, organismo politico-tecnico coordinato dal CIdiS. Detto Tavolo ha attivato la realizzazione di una mappatura delle attivitа, progetti e servizi rivolti ai bambini, ai giovani e alle famiglie, in relazione alle aree ambiente e sport, comunicazione, comunitа educante, governance, misure economiche e welfare territoriale. Nell’ambito di questo percorso l’Amministrazione comunale si pone, dunque, fra gli obiettivi, quello di vedersi assegnare il marchio “Comune Amico della Famiglia” e l’annessa certificazione in riferimento a tutta l’attivitа ordinaria e straordinaria svolta a sostegno delle politiche per il benessere della comunitа e delle famiglie e come previsto dagli indicatori della Provincia autonoma di Trento, il percorso virtuoso che si intende intraprendere prevede la redazione del seguente Piano di interventi in materia di politiche familiari per l’anno 2022 le cui misure sono state approvate con la Deliberazione della Giunta Comunale n. 13 del 20 gennaio 2022.





=== AZIONI ===



TITOLO: Tavolo delle Associazioni Comunali

TASSONOMIA: Incontri di progettazione e programmazione, gruppo di lavoro

OBIETTIVO: Promuovere il Progetto Family al fine di stimolare la nascita di nuove iniziative rivolte alle famiglie da parte delle stesse Associazioni del territorio.

DESCRIZIONE: Istituzione del Tavolo delle Associazioni Comunali 

-----



TITOLO: Partecipazione al Network Comuni Amici Della Famiglia

TASSONOMIA: Adesione  ai marchi familiari

OBIETTIVO: Promuovere il confronto sulle buone prassi e iniziative, in favore delle famiglie e della

comunitа, attivate da altri Comuni Family aderenti al Network. 

DESCRIZIONE: Adesione al Network nazionale Comuni amici della famiglia 

-----



TITOLO: Iniziative in accordo con esercizi di commerciali e Associazioni

TASSONOMIA: Sistemi premianti

OBIETTIVO: L’Amministrazione intende stimolare gli esercizi commerciali e le Associazioni esistenti sul

territorio a collaborare con il Comune predisponendo offerte e sconti volti a sostenere le

famiglie numerose, creando altresм momenti di incontro per spiegare loro le iniziative del

progetto Family.

DESCRIZIONE: Attivazione di collaborazioni con esercizi commerciali e Associazioni 

-----



TITOLO: Visibilitа al marchio Family

TASSONOMIA: Promozione strumenti ACS (Family Audit, Family in Trentino, Network nazionale, Distretto Famiglia, EuregioFamilyPass, Family in Italia)

OBIETTIVO: L’Amministrazione comunale intende dare visibilitа al marchio Family, inserendo il logo

sulla porta del Comune, nella carta intestata, nei volantini dei progetti dedicati alle famiglie,

etc.

DESCRIZIONE: Promozione del marchio connesso alle certificazioni Family  

-----



TITOLO: Sostegno ad Associazioni del territorio che aiutano le famiglie

TASSONOMIA: Sostegno economico alle associazioni del territorio / Concessione spazi

OBIETTIVO: L’Amministrazione intende continuare a sostenere con un contributo le Associazioni che

forniscono aiuti economici alle famiglie in difficoltа nel pagamento delle bollette, dell'affitto

e delle spese condominiali.

DESCRIZIONE: Contributo economico alle Associazioni 

-----



TITOLO: Concessione di un contributo a famiglie non assegnatarie di  posto al nido

TASSONOMIA: Agevolazioni tariffarie e contributi per servizi educativi e di conciliazione

OBIETTIVO: L'Amministrazione ha previsto l’erogazione di un contributo a favore delle famiglie che,

seppur utilmente collocate nella graduatoria di accesso all'asilo comunale, non sono

individuate quali assegnatarie. Il contributo è vincolato all’effettiva iscrizione a specifici

servizi educativi offerti sul territorio.

DESCRIZIONE: Contributo economico a famiglie 

-----



TITOLO: Festa dei nuovi nati

TASSONOMIA: Promozione della natalitа (Bonus bebè, kit nuovi nati ecc.)

DESCRIZIONE: L'Amministrazione intende proseguire ed integrare, per l'anno 2022, la Festa dei nuovi nati

(manifestazione rientrante nel più ampio progetto ''Nati per leggere'') rivolta alle famiglie

con bimbi nati al 31 dicembre 2021. Tali famiglie verranno invitate ed accolte dal Sindaco

presso la Biblioteca con un’attivitа ludica/educativa adatta ai bimbi piccoli ed un “pacchetto

di benvenuto” contenente un libro per l'infanzia.

-----



TITOLO: Presenza di Wi-Fi pubblico nel Comune

TASSONOMIA: Wifi free

DESCRIZIONE: L’Amministrazione intende promuovere “Piazza Wi-fi Italia”, un progetto del Ministero dello

Sviluppo Economico che ha come obiettivo di permettere a tutti i cittadini di connettersi,

gratuitamente e in modo semplice tramite l’App dedicata, a una rete wi-fi libera e diffusa su

tutto il territorio nazionale.

-----



TITOLO: Sostegno alle Associazioni che promuovono iniziative rivolte alle famiglie

TASSONOMIA: Sostegno economico alle associazioni del territorio / Concessione spazi

DESCRIZIONE: L'Amministrazione intende sostenere e promuovere sul territorio le Associazioni

nell’organizzazione di eventi che abbiamo un’attenzione particolare rivolta alle famiglie

con allestimenti di spazi gioco, svago e intrattenimento dei bambini durante fiere, sagre o

eventi vari.

-----



TITOLO: Pagina Web dedicata al progetto Family

TASSONOMIA: Strumenti di comunicazione

DESCRIZIONE: Verrа inserita sul sito web ufficiale del Comune una pagina dedicata al progetto Family, per

spiegare il progetto con le sue finalitа e tutte le azioni che verranno intraprese nel tempo.

-----



TITOLO: Calendario condiviso tra tutte le Associazioni del Territorio

TASSONOMIA: Strumenti di comunicazione

DESCRIZIONE: L’Amministrazione intende attivare un calendario, disponibile sulla pagina web del

Comune, per rendere visibili tutte le iniziative relative al progetto Family

-----



TITOLO: Sportello Family

TASSONOMIA: Sportelli informativi e di orientamento

DESCRIZIONE: L’Amministrazione intende attivare uno “Sportello della Famiglia”, inteso come luogo

virtuale/help desk in cui le famiglie possano reperire tutte le informazioni di cui

necessitano. Lo Sportello Famiglia metterа a disposizione informazioni relative ai servizi

d'interesse per le famiglie, tra cui l’orientamento relativo a servizi sociali, iniziative, bandi e

agevolazioni.

-----



TITOLO: Carnevale dei bambini

TASSONOMIA: Promozione e organizzazione di eventi ludici (festa delle famiglie, spettacoli ecc.)

OBIETTIVO: Realizzare un momento di incontro e di svago per le famiglie del territorio in occasione del  Carnevale.

DESCRIZIONE: L'Amministrazione intende ripristinare, nel rispetto delle necessarie misure di sicurezza igienico-sanitarie, il “Carnevale dei bambini”, una manifestazione dedicata all'intrattenimento dei bambini in occasione del Carnevale (con sfilate in maschera e giochi dedicati).

-----



TITOLO: Alcool Free Day

TASSONOMIA: Azioni di contrasto alle dipendenze

OBIETTIVO: Sensibilizzare i giovani alle conseguenze dovute all’abuso di sostanze stupefacenti e alcool.

DESCRIZIONE: L'Amministrazione intende promuovere, presso gli spazi della biblioteca ed all'interno degli orari di apertura della stessa, incontri volti a sensibilizzare i giovani sulle gravi conseguenze legate al consumo di sostanze stupefacenti e alcool.

-----



TITOLO: Spazio compiti

TASSONOMIA: Servizi doposcuola e servizi estivi

DESCRIZIONE: Fornire sostegno alle famiglie nella gestione delle attivitа pomeridiane e favorire l’aggregazione al di fuori degli spazi scolastici.

-----



TITOLO: Spazio allattamento

TASSONOMIA: Fasciatoi/Spazio allattamento/Baby Little Home/Spazi per la famiglia

DESCRIZIONE: L'Amministrazione vorrebbe attivare, presso la biblioteca comunale una zona munita di

poltrona e paravento ed attrezzata con scalda biberon, tappeto e giochi rivolto a

qualunque mamma/papа che abbia la necessitа di allattare o scaldare il biberon per il

proprio bimbo.

-----



TITOLO: Fasciatoio

TASSONOMIA: Fasciatoi/Spazio allattamento/Baby Little Home/Spazi per la famiglia

DESCRIZIONE: L’Amministrazione vorrebbe predisporre, presso Palazzo Civico, un bagno-famiglia dotato

di fasciatoio per il cambio pannolino.

-----



TITOLO: Parcheggi rosa e parcheggi family

TASSONOMIA: Sostegno alla mobilitа

DESCRIZIONE: Individuazione di nuove aree per i “parcheggi rosa”/“parcheggi family” da riservare a

donne in gravidanza o mamme e papа con bimbi fino a due anni di etа.

-----



TITOLO: Parco inclusivo

TASSONOMIA: Pianificazione urbanistica (strade, parchi, orti, campi sportivi ecc.)

OBIETTIVO: Garantire uno spazio dove tutti i bambini possano giocare e muoversi “senza barriere” ,

attraverso attrezzature ludiche utilizzabili anche dalla maggior parte dei bambini con

disabilitа. 

DESCRIZIONE: L'Amministrazione intende promuovere l'utilizzo del parco giochi comunale inclusivo

realizzato all'interno del “Giardino della Memoria”, sito in Via A. De Gasperi (D.G. n. 80 del

23 giugno 2021).
"""

In [ ]:
interactive_prompt = """
You are an helpful assistant from the public administration, use a strictly {{STYLING}} tone.
Your task is to provide a relevant answer to the user using the provided evidence and past dialogue history.
The evidence is contained .
Use the document between <document> XML tags only as reference, do not mention in the conversation.
Answer in Italian.
When you receive a question, answer by referring exclusively to the content of the document.
The user is a {{USER_TYPE}}.

<document>{{DOCUMENT}}</document>
"""

dial_styling = "Informal" # Formal | Informal

user_type = "Public Operator" # Public Operator | Citizen

In [74]:
llama_interactive_dialogue(interactive_prompt\
    .replace("{{DOCUMENT}}", interactive_doc)\
    .replace("{{STYLING}}", dial_styling)\
    .replace("{{USER_TYPE}}", user_type))

         You are starting a conversation            
- - - - - - - - - - - - - - - - - - - - - - - - - - 
> to interrupt the chat, write "stop" or "exit" 
> to save the chat on the disk, write "save"      
- - - - - - - - - - - - - - - - - - - - - - - - - - 

USER:
Mio figlio è disabile, che servizi vengono offerti riguardo la sua educazione?

MODEL:   (wait time: 8.3 seconds)                 
Il Comune di Milano offre un servizio di assistenza educativa che fa parte del progetto educativo individualizzato e si distingue e completa l'attività didattica del docente di sostegno e l'assistenza di base garantita dal personale scolastico ausiliario. Il servizio garantisce ai disabili il diritto allo studio e lo sviluppo delle potenzialità nell'apprendimento, nella comunicazione, nelle relazioni e nella socializzazione attraverso un percorso di integrazione scolastica.

USER:
Mio figlio è residente nel comune di Busto Arsizio. Può accedere anche lui a questo servizio?

MODEL:   (wait time: 9